In [1]:
import sys
sys.path.append("..") ## resetting the path to the parent directory

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, load_model

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
from sklearn.metrics import classification_report

In [5]:
import numpy as np
import pandas as pd

In [6]:
model = load_model('../models/vgg_v3_partial_unfreeze.h5')

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              25691136  
_________________________________________________________________
predictions (Dense)          (None, 12)                12300     
Total params: 40,418,124
Trainable params: 32,782,860
Non-trainable params: 7,635,264
_________________________________________________________________


In [8]:
eval_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [11]:
batch_size=32
train_generator = train_datagen.flow_from_directory(
        '../data/train_folder',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 150x150
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

Found 10285 images belonging to 12 classes.


In [12]:
evaluate_generator = eval_datagen.flow_from_directory(
        '../data/validation_folder',
        target_size=(224, 224),
        batch_size=1,
        seed=42,
        shuffle=False,
        class_mode='categorical')

Found 1140 images belonging to 12 classes.


In [13]:
test_generator = test_datagen.flow_from_directory(
    directory='../data/holdout_folder',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    shuffle=False,
    class_mode=None,
    seed=42)

Found 1371 images belonging to 12 classes.


In [14]:
eval_loss, eval_acc = model.evaluate(evaluate_generator, verbose=0)

In [15]:
print(f'The validation accuracy is: {eval_acc*100:.2f}%')

The validation accuracy is: 83.07%


In [16]:
pred = model.predict(test_generator,verbose=1)

1371/1371 [==============================] - 44s 32ms/step


In [17]:
y_pred = np.argmax(pred, axis=1)

In [18]:
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

In [19]:
report = classification_report(y_true, y_pred, target_names=class_labels)

In [20]:
print(report)

                 precision    recall  f1-score   support

 Antoninus Pius       0.85      0.90      0.87       123
  Constantine I       0.76      0.84      0.80       116
 Constantine II       0.76      0.65      0.70       129
 Constantius II       0.74      0.81      0.77       118
     Diocletian       0.87      0.82      0.84        92
       Galerius       0.76      0.79      0.78       114
      Gallienus       0.96      0.98      0.97       140
        Hadrian       0.81      0.84      0.83       102
       Licinius       0.85      0.82      0.84       120
Marcus Aurelius       0.91      0.83      0.87       116
       Maximian       0.77      0.80      0.78        88
         Trajan       0.95      0.92      0.94       113

       accuracy                           0.83      1371
      macro avg       0.83      0.83      0.83      1371
   weighted avg       0.84      0.83      0.83      1371



In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
sl_y_true = {value: key for key,value in test_generator}

SyntaxError: invalid syntax (<ipython-input-22-830c00217790>, line 1)

In [ ]:
y_true

In [ ]:
cm_y_true = [sl_y_true[val]for val in y_true]

In [ ]:
cm_y_pred = [sl_y_true[val] for val in y_pred]

In [ ]:
cm = confusion_matrix(y_true,y_pred)
print(cm)

In [ ]:
%load_ext autoreload
#Satutoreload 2

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')


In [ ]:
sns.heatmap(cm)

In [ ]:
%load_ext autoreload
#Thuutoreload 2

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

cm2 = confusion_matrix(cm_y_true,cm_y_pred)
labels = [val for val in sl_y_true.values()]
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
ax = plt.subplot()
sns.heatmap(cm2)
ax.xaxis.set_ticklabels([val for val in sl_y_true.values()]); 
ax.yaxis.set_ticklabels([val for val in sl_y_true.values()][::-1]);
